# **Food/Fruit Recognition and Calorie Estimation**
## **Part C Fruit Classification**

## **Imports**

In [7]:
import os
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import (
    ResNet50,
    EfficientNetB4,
    ConvNeXtTiny
)
import matplotlib.pyplot as plt


## **Configuration**

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

class Config:
    TRAIN_DIR = "/kaggle/input/vision-data/Project Data/Fruit/Train"
    VAL_DIR   = "/kaggle/input/vision-data/Project Data/Fruit/Validation"

    IMG_SIZE = 350
    BATCH_SIZE = 14
    EPOCHS = 10
    LR = 1e-4


## **DataLoaders**

In [9]:
def load_image_paths(root_dir):
    image_paths = []
    labels = []
    class_names = sorted(os.listdir(root_dir))

    class_to_idx = {cls: i for i, cls in enumerate(class_names)}

    for cls in class_names:
        img_dir = os.path.join(root_dir, cls, "Images")
        if not os.path.isdir(img_dir):
            continue

        for f in os.listdir(img_dir):
            if f.lower().endswith((".jpg", ".png", ".jpeg")):
                image_paths.append(os.path.join(img_dir, f))
                labels.append(class_to_idx[cls])

    return image_paths, labels, class_names


train_paths, train_labels, class_names = load_image_paths(Config.TRAIN_DIR)
val_paths, val_labels, _ = load_image_paths(Config.VAL_DIR)

NUM_CLASSES = len(class_names)



## **Preprocessing**

In [10]:
MEAN = tf.constant([0.485, 0.456, 0.406])
STD  = tf.constant([0.229, 0.224, 0.225])

def load_and_preprocess(path, label, augment=False):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (Config.IMG_SIZE, Config.IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0

    if augment:
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.7, 1.3)
        img = tf.image.rot90(img, tf.random.uniform([], 0, 4, tf.int32))

    img = (img - MEAN) / STD
    return img, label



AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.shuffle(1000)
train_ds = train_ds.map(
    lambda x, y: load_and_preprocess(x, y, augment=True),
    num_parallel_calls=AUTOTUNE
)
train_ds = train_ds.batch(Config.BATCH_SIZE).prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = val_ds.map(
    lambda x, y: load_and_preprocess(x, y, augment=False),
    num_parallel_calls=AUTOTUNE
)
val_ds = val_ds.batch(Config.BATCH_SIZE).prefetch(AUTOTUNE)


## **ResNet50**

In [14]:
base = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(Config.IMG_SIZE, Config.IMG_SIZE, 3)
)

inputs = layers.Input((Config.IMG_SIZE, Config.IMG_SIZE, 3))
x = base(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)



model.compile(
    optimizer=tf.keras.optimizers.Adam(Config.LR),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

start_time = time.time()

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=Config.EPOCHS
)
end_time = time.time()

model.save("ResNet50.keras")

# 5. Calculate and Print the Training Time
total_training_time_seconds = end_time - start_time
minutes = int(total_training_time_seconds // 60)
seconds = int(total_training_time_seconds % 60)
print("--------------------------------------------------")
print(f"✅ Training Finished. Model saved to ResNet50.keras")
print(f"⏱️ Total Training Time: {minutes} minutes and {seconds} seconds ({total_training_time_seconds:.2f} seconds)")
print("--------------------------------------------------")

Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 132s 580ms/step - accuracy: 0.4443 - loss: 2.2524 - val_accuracy: 0.0333 - val_loss: 5.1804
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 41s 322ms/step - accuracy: 0.9098 - loss: 0.3498 - val_accuracy: 0.0600 - val_loss: 5.6014
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 40s 318ms/step - accuracy: 0.9692 - loss: 0.1400 - val_accuracy: 0.1800 - val_loss: 3.5313
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 40s 318ms/step - accuracy: 0.9714 - loss: 0.1029 - val_accuracy: 0.6133 - val_loss: 1.6994
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 40s 319ms/step - accuracy: 0.9769 - loss: 0.0989 - val_accuracy: 0.9067 - val_loss: 0.5190
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 40s 318ms/step - accuracy: 0.9764 - loss: 0.0707 - val_accuracy: 0.9733 - val_loss: 0.0601
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 40s 320ms/step - accuracy: 0.9878 - loss: 0.0562 - val_accuracy: 0.9667 - val_loss: 0.1350
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 40s 319ms/step - accuracy: 0.9610 - loss: 

## **EfficientNet-B4**

In [13]:
base = EfficientNetB4(
    weights="imagenet",
    include_top=False,
    input_shape=(Config.IMG_SIZE, Config.IMG_SIZE, 3)
)

inputs = layers.Input((Config.IMG_SIZE, Config.IMG_SIZE, 3))
x = base(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)


model.compile(
    optimizer=tf.keras.optimizers.Adam(Config.LR),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

start_time = time.time()
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=Config.EPOCHS
)
end_time = time.time()
model.save("EfficientNetB4.keras")


# 5. Calculate and Print the Training Time
total_training_time_seconds = end_time - start_time
minutes = int(total_training_time_seconds // 60)
seconds = int(total_training_time_seconds % 60)
print("--------------------------------------------------")
print(f"✅ Training Finished. Model saved to EfficientNetB4.keras")
print(f"⏱️ Total Training Time: {minutes} minutes and {seconds} seconds ({total_training_time_seconds:.2f} seconds)")
print("--------------------------------------------------")

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


E0000 00:00:1765884988.310596     441 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884988.475853     441 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884997.106596     441 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884997.249033     441 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884997.598672     441 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

125/126 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.3150 - loss: 2.7078

E0000 00:00:1765885097.832827     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765885097.986532     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765885104.560337     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765885104.698457     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765885105.014129     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 847ms/step - accuracy: 0.3163 - loss: 2.7035

E0000 00:00:1765885155.318762     441 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765885155.475065     441 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


126/126 ━━━━━━━━━━━━━━━━━━━━ 260s 998ms/step - accuracy: 0.3177 - loss: 2.6992 - val_accuracy: 0.0667 - val_loss: 3.2181
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 48s 376ms/step - accuracy: 0.8788 - loss: 0.6616 - val_accuracy: 0.5200 - val_loss: 1.8685
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 47s 371ms/step - accuracy: 0.9599 - loss: 0.2074 - val_accuracy: 0.8667 - val_loss: 0.5016
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 47s 373ms/step - accuracy: 0.9791 - loss: 0.1154 - val_accuracy: 0.9933 - val_loss: 0.0529
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 47s 371ms/step - accuracy: 0.9784 - loss: 0.0970 - val_accuracy: 0.9667 - val_loss: 0.1092
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 47s 373ms/step - accuracy: 0.9814 - loss: 0.0758 - val_accuracy: 1.0000 - val_loss: 0.0151
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 47s 373ms/step - accuracy: 0.9890 - loss: 0.0525 - val_accuracy: 1.0000 - val_loss: 0.0064
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 47s 373ms/step - accuracy: 0.9961 - loss: 0.0287 - va

## **ConvNeXT**

In [12]:
base = ConvNeXtTiny(
    weights="imagenet",
    include_top=False,
    input_shape=(Config.IMG_SIZE, Config.IMG_SIZE, 3)
)

inputs = layers.Input((Config.IMG_SIZE, Config.IMG_SIZE, 3))
x = base(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.LayerNormalization()(x)
x = layers.Dense(512, activation="gelu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)


model.compile(
    optimizer=tf.keras.optimizers.Adam(Config.LR),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
start_time = time.time()
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=Config.EPOCHS
)
end_time = time.time()
model.save("ConvNeXtTiny.keras")



total_training_time_seconds = end_time - start_time
minutes = int(total_training_time_seconds // 60)
seconds = int(total_training_time_seconds % 60)
print("--------------------------------------------------")
print(f"✅ Training Finished. Model saved to ConvNeXtTiny.keras")
print(f"⏱️ Total Training Time: {minutes} minutes and {seconds} seconds ({total_training_time_seconds:.2f} seconds)")
print("--------------------------------------------------")

111650432/111650432 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


E0000 00:00:1765884115.333307     442 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884115.468445     442 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884117.628190     442 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884117.763680     442 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


125/126 ━━━━━━━━━━━━━━━━━━━━ 0s 574ms/step - accuracy: 0.3077 - loss: 2.6159

E0000 00:00:1765884210.589284     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884210.724403     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884212.519004     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884212.653649     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 746ms/step - accuracy: 0.3096 - loss: 2.6087

E0000 00:00:1765884233.390228     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765884233.525361     444 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


126/126 ━━━━━━━━━━━━━━━━━━━━ 152s 813ms/step - accuracy: 0.3115 - loss: 2.6016 - val_accuracy: 0.9067 - val_loss: 0.3754
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 74s 584ms/step - accuracy: 0.9432 - loss: 0.2199 - val_accuracy: 0.9667 - val_loss: 0.1361
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 73s 578ms/step - accuracy: 0.9778 - loss: 0.0890 - val_accuracy: 1.0000 - val_loss: 0.0383
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 73s 577ms/step - accuracy: 0.9756 - loss: 0.0938 - val_accuracy: 0.9733 - val_loss: 0.0763
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 73s 580ms/step - accuracy: 0.9837 - loss: 0.0633 - val_accuracy: 1.0000 - val_loss: 0.0138
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 73s 580ms/step - accuracy: 0.9919 - loss: 0.0284 - val_accuracy: 1.0000 - val_loss: 0.0215
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 73s 580ms/step - accuracy: 0.9865 - loss: 0.0466 - val_accuracy: 0.9933 - val_loss: 0.0288
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 73s 581ms/step - accuracy: 0.9881 - loss: 0.0481 - va

## **MobileNetV2**

In [11]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

IMG_SIZE = Config.IMG_SIZE
NUM_CLASSES = len(class_names) 

base_mobilenet = MobileNetV2(
    weights='imagenet',               
    include_top=False,                 
    input_shape=(IMG_SIZE, IMG_SIZE, 3)  
)

base_mobilenet.trainable = False


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_mobilenet(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)             
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(NUM_CLASSES, activation='softMax')(x)


model_mobilenet = models.Model(inputs, outputs)


model_mobilenet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=Config.LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

start_time = time.time()
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=Config.EPOCHS
)
end_time = time.time()
model.save("MobileNetV2.keras")



total_training_time_seconds = end_time - start_time
minutes = int(total_training_time_seconds // 60)
seconds = int(total_training_time_seconds % 60)
print("--------------------------------------------------")
print(f"✅ Training Finished. Model saved to MobileNetV2.keras")
print(f"⏱️ Total Training Time: {minutes} minutes and {seconds} seconds ({total_training_time_seconds:.2f} seconds)")
print("--------------------------------------------------")

/tmp/ipykernel_47/3321348109.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_mobilenet = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


E0000 00:00:1765912890.336794     124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765912890.505797     124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765912902.969360     124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765912903.111777     124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765912903.521000     124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

125/126 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 1.0000 - loss: 0.0048

E0000 00:00:1765913009.294732     126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765913009.448474     126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765913016.326261     126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765913016.464090     126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765913016.770335     126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 894ms/step - accuracy: 1.0000 - loss: 0.0048

E0000 00:00:1765913064.984819     126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765913065.142158     126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


126/126 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 1.0000 - loss: 0.0048 - val_accuracy: 1.0000 - val_loss: 1.5080e-04
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 56s 445ms/step - accuracy: 0.9973 - loss: 0.0093 - val_accuracy: 0.9933 - val_loss: 0.0076
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 54s 430ms/step - accuracy: 0.9989 - loss: 0.0092 - val_accuracy: 1.0000 - val_loss: 0.0126
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 55s 433ms/step - accuracy: 0.9979 - loss: 0.0102 - val_accuracy: 0.9533 - val_loss: 0.1988
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 55s 435ms/step - accuracy: 0.9984 - loss: 0.0072 - val_accuracy: 0.9800 - val_loss: 0.0716
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 55s 434ms/step - accuracy: 0.9985 - loss: 0.0089 - val_accuracy: 0.9867 - val_loss: 0.0231
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 55s 434ms/step - accuracy: 0.9983 - loss: 0.0054 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 55s 437ms/step - accuracy: 0.9984 - loss: 0.0080 - v

## **Custom CNN**

In [24]:
# ============================================================
# CUSTOM CNN MODEL (FROM SCRATCH) - TENSORFLOW
# ============================================================

import time
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# -----------------------------
# MODEL ARCHITECTURE
# -----------------------------
def build_custom_cnn(input_shape, num_classes):
    model = models.Sequential(name="CustomCNN")

    # Block 1
    model.add(layers.Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 2
    model.add(layers.Conv2D(64, (3, 3), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 3
    model.add(layers.Conv2D(128, (3, 3), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 4
    model.add(layers.Conv2D(256, (3, 3), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2)))

    # Classification Head
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation="softmax"))

    return model


# -----------------------------
# BUILD MODEL
# -----------------------------
input_shape = (Config.IMG_SIZE, Config.IMG_SIZE, 3)
num_classes = len(class_names)

custom_cnn = build_custom_cnn(input_shape, num_classes)

custom_cnn.summary()


# -----------------------------
# COMPILE
# -----------------------------
custom_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(Config.LR),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


# -----------------------------
# CALLBACKS
# -----------------------------
custom_callbacks = [
    callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        restore_best_weights=True
    ),
    callbacks.ReduceLROnPlateau(
        monitor="val_accuracy",
        factor=0.5,
        patience=5,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        "CustomCNN_best.keras",
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    )
]


# -----------------------------
# TRAINING 
# -----------------------------
start_time = time.time()
EPOCHS=30
history_custom = custom_cnn.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=custom_callbacks
)

training_time_custom = time.time() - start_time


# -----------------------------
# FINAL SAVE
# -----------------------------
custom_cnn.save("CustomCNN_final.keras")

print(f"\n⏱ Training Time (Custom CNN): {training_time_custom/60:.2f} minutes")


Model: "CustomCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 350, 350, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 350, 350, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_8 (ReLU)                  │ (None, 350, 350, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 175, 175, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 175, 175, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 175, 175, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_9 (ReLU)                  │ (None, 175, 175, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 87, 87, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 87, 87, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 87, 87, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_10 (ReLU)                 │ (None, 87, 87, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 43, 43, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 43, 43, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 43, 43, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_11 (ReLU)                 │ (None, 43, 43, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 21, 21, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 112896)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1024)           │   115,606,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 30)             │        15,390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 116,537,054 (444.55 MB)

 Trainable params: 116,536,094 (444.55 MB)

 Non-trainable params: 960 (3.75 KB)

Epoch 1/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.1684 - loss: 8.7664
Epoch 1: val_accuracy improved from -inf to 0.08667, saving model to CustomCNN_best.keras
126/126 ━━━━━━━━━━━━━━━━━━━━ 30s 184ms/step - accuracy: 0.1686 - loss: 8.7448 - val_accuracy: 0.0867 - val_loss: 3.1135 - learning_rate: 1.0000e-04
Epoch 2/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.3126 - loss: 2.5818
Epoch 2: val_accuracy improved from 0.08667 to 0.24000, saving model to CustomCNN_best.keras
126/126 ━━━━━━━━━━━━━━━━━━━━ 18s 145ms/step - accuracy: 0.3124 - loss: 2.5817 - val_accuracy: 0.2400 - val_loss: 2.7835 - learning_rate: 1.0000e-04
Epoch 3/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.3755 - loss: 2.2393
Epoch 3: val_accuracy improved from 0.24000 to 0.38667, saving model to CustomCNN_best.keras
126/126 ━━━━━━━━━━━━━━━━━━━━ 19s 146ms/step - accuracy: 0.3754 - loss: 2.2395 - val_accuracy: 0.3867 - val_loss: 1.8406 - learning_rate: 1.0000e-04
Epoch 4/30
126/126

# ***Prepared By Team 3***